In [ ]:
from Trans_Tokenize import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from sklearn.model_selection import train_test_split
import math
import unicodedata
import re
import swifter
import nltk

In [16]:
raw_data=pd.read_csv('../dataset/job_data_new.csv')

In [17]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26361 entries, 0 to 26360
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Job Name              26361 non-null  object
 1   Job Link              26361 non-null  object
 2   Location              26361 non-null  object
 3   Update Date           26361 non-null  object
 4   Deadline              26361 non-null  object
 5   Experience            26361 non-null  object
 6   Job Level             26361 non-null  object
 7   Industry              26361 non-null  object
 8   Employment Type       26359 non-null  object
 9   Welfare               26361 non-null  object
 10  Job Description       26361 non-null  object
 11  Job Requirements      26361 non-null  object
 12  Other Information     26361 non-null  object
 13  Job Tags              26361 non-null  object
 14  Company URL           26361 non-null  object
 15  Salary                26361 non-null

In [18]:
used_data=raw_data[["Location","Experience","Job Level","Employment Type" ,"Industry","Welfare","Job Description","Job Requirements","Other Information","Follower","Salary"]]

In [19]:
used_data.head()

,Location,Experience,Job Level,Employment Type,Industry,Welfare,Job Description,Job Requirements,Other Information,Follower,Salary
0,Hồ Chí Minh,3 - 5 \n ...,Quản lý,Nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Phụ cấp, Đồng phục, Chế độ th...",Tuyển dụng và đào tạo: Quản lý quy trình tuyển...,Trình độ: Tốt nghiệp cao đẳng/đại học chuyên n...,Bằng cấp:\n Đại học\nThời gian ...,NaN,Cạnh tranh
1,Hồ Chí Minh,Not specified,Nhân viên,Nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Phụ cấp, Xe đưa đón, Đồng phụ...","Thương lượng với khách hàng về giá cả, chất lư...","Tốt nghiệp đại học các chuyên ngành Kinh tế, n...",Bằng cấp:\n Đại học\nĐộ tuổi:\n...,NaN,Cạnh tranh
2,Not specified,Not specified,Not specified,Not specified,Not Found,"Chế độ bảo hiểm, Phụ cấp, Xe đưa đón, Đào tạo,...",Not specified,Not specified,Bằng cấp:\n Trung cấp\nHình thứ...,NaN,Not specified
3,Bình Định,2 - 5 \n ...,Trưởng nhóm / Giám sát,Nhân viên chính thức,\n ...,"Laptop, Chế độ bảo hiểm, Du Lịch, Phụ cấp, Xe ...",Job Description:\n1、Develop and optimize depar...,Experience Requirement：\n- Over 3 years...,Bằng cấp:\n Đại học\nĐộ tuổi:\n...,169,Cạnh tranh
4,Hà Nội,Trên 1 \n ...,Nhân viên,Nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Du Lịch, Chế độ thưởng, Chăm ...",- Phát triển khách hàng; tìm kiếm khách hàng ...,- Tốt nghiệp Đại Học hệ chính qui thuộc các n...,Độ tuổi:\n Không giới hạn tuổi\...,2.27,Cạnh tranh


### Location

In [20]:
location_data=used_data.copy()


In [21]:
def remove_diacritics(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )

In [22]:
location_data['Location'] = location_data['Location'].apply(lambda x: x.title())

location_data['Location'] = location_data['Location'].apply(lambda x: x.split(',')[0].strip() if ',' in x else x.strip())
location_data['Location'] = location_data['Location'].apply(remove_diacritics)

In [23]:
city_to_number = {
    "Hai Duong": 0,
    "Long An": 1,
    "Ha Noi": 2,
    "Ho Chi Minh": 3,
    "Binh Duong": 4,
    "Đong Nai": 5,
    "Quang Ngai": 6,
    "Vinh Long": 7,
    "Lam Đong": 8,
    "Can Tho": 9,
    "Bac Ninh": 10,
    "Dak Lak": 11,
    "Gia Lai": 12,
    "Bac Giang": 13,
    "Binh Thuan": 14,
    "Tay Ninh": 15,
    "Ba Ria - Vung Tau": 16,
    "Ha Nam": 17,
    "Thai Binh": 18,
    "Bac Lieu": 19,
    "Thanh Hoa": 20,
    "Binh Đinh": 21,
    "Tra Vinh": 22,
    "Hoa Binh": 23,
    "Đa Nang": 24,
    "Hung Yen": 25,
    "Hai Phong": 26,
    "Kien Giang": 27,
    "Vinh Phuc": 28,
    "Ha Giang": 29,
    "Tuyen Quang": 30,
    "Đien Bien": 31,
    "Nam Dinh": 32,
    "Lang Son": 33,
    "Nghe An": 34,
    "Quang Tri": 35,
    "Quang Ninh": 36,
    "Ninh Thuan": 37,
    "Đong Thap": 38,
    "Quang Nam": 39,
    "Thua Thien- Hue": 40,
    "Tien Giang": 41,
    "Khanh Hoa": 42,
    "Phu Yen": 43,
    "Quang Binh": 44,
    "Hau Giang": 45,
    "Binh Phuoc": 46,
    "Phu Tho": 47,
    "Ninh Binh": 48,
    "Thai Nguyen": 49,
    "Bac Can": 50,
    "Son La": 51,
    "Ha Tinh": 52,
    "Yen Bai": 53,
    "Ca Mau": 54,
    "An Giang": 55,
    "Ben Tre": 56,
    "Lao Cai": 57,
    "Soc Trang": 58,
    "Cao Bang": 59,
    "Dak Nong": 60,
    "Kon Tum": 61,
    "Lai Chau": 62,
    "Nam Đinh":63,
    
    # Khu vực khác
    "Kv Đong Nam Bo": 64,
    "Kv Tay Nguyen": 65,
    "Kv Nam Trung Bo": 66,
    "Kv Bac Trung Bo": 67,
    "Đong Bang Song Cuu Long": 68,
    "Toan Quoc": 69,
    "Khac": 70,
    "Not Specified": 70,

    # Các địa điểm khác (được gộp chung vào "foreign city/province")
    "Svay Rieng": 71,
    "Tokyo": 72,
    "Xiangkhouang": 73,
    "Champasak": 74,
    "Yokohama": 75,
    "Hokkaido": 76,
    "Malaysia": 77,
    "Attapeu":78,
    "Bangkok": 79,
    "Kratie": 80,
    "Phnompenh": 81,
    "Vientiane":82,
    "Kuala Lumpur": 83,
    "Quoc Te": 84
}

In [24]:
location_data['Location'] = location_data['Location'].replace(city_to_number)

In [25]:
location=location_data['Location'].unique()
location

array([ 3, 70, 21,  2,  0, 55,  1, 36, 24,  4,  6,  5, 25,  7, 17,  8,  9,
       10, 11, 12, 42, 13, 14, 15, 16, 41, 18, 19, 20, 22, 37, 23, 46, 26,
       27, 59, 53, 45, 28, 29, 30, 31, 63, 33, 64, 34, 35, 78, 38, 39, 40,
       69, 43, 44, 82, 47, 56, 71, 48, 49, 50, 51, 52, 60, 54, 65, 57, 58,
       68, 84, 61, 72, 73, 79, 74, 75, 76, 77, 67, 80, 66, 62, 81, 83],
      dtype=int64)

### Experience

In [26]:
experience_data=location_data.copy()

In [27]:
def extract_number(value):
    if 'Chưa có kinh nghiệm' in value:
        return 0  # Nếu "Chưa có kinh nghiệm" trả về 0
    elif match := re.match(r'(\d+)\s*-\s*\d+', value):  # Nếu định dạng là number - number
        return int(match.group(1))  # Lấy số đầu tiên
    elif match := re.search(r'\d+', value):  # Nếu chỉ có một số duy nhất
        return int(match.group(0))  # Lấy số đó
    return 0

In [28]:
experience_data["Experience"]=experience_data["Experience"].str.replace("\n","",regex=True)
experience_data["Experience"]=experience_data["Experience"].str.replace("Năm","",regex=True)
experience_data["Experience"]=experience_data["Experience"].str.replace(r"\s+",' ',regex=True).str.strip()
experience_data["Experience"]=experience_data["Experience"].apply(extract_number)

In [29]:
experience=experience_data["Experience"].unique()
experience

array([ 3,  0,  2,  1,  5,  4,  7, 10,  8,  6, 15,  9, 12, 30, 20],
      dtype=int64)

### Job Level

In [30]:
job_level_data=experience_data.copy()



In [31]:
job_level_list = {
    'mới tốt nghiệp':1,
    'sinh viên/ thực tập sinh':1,
    'nhân viên':2,
    'trưởng nhóm':3,
    'trưởng nhóm / giám sát':3,
    'quản lý':4,
    'phó giám đốc':5,
    'giám đốc':6,
    'tổng giám đốc':7,
    'not specified':0
}

job_level_data['Job Level'] = job_level_data['Job Level'].str.lower()

job_level_data['Job Level'] = job_level_data['Job Level'].map(job_level_list)

In [32]:
job_level=job_level_data["Job Level"].unique()
job_level

array([4, 2, 0, 3, 6, 1, 5, 7], dtype=int64)

### Employment Type

In [33]:
employment_type_data=job_level_data.copy()

In [34]:
employment_type_data["Employment Type"]=employment_type_data["Employment Type"].str.lower()
employment_type_data["Employment Type"]=employment_type_data["Employment Type"].fillna("not specified")

employment=employment_type_data["Employment Type"].unique()
employment=pd.DataFrame(employment,columns=["Employment Type"])
employment = employment.dropna()
employment = employment[employment["Employment Type"] != "not specified"]

split_job_types = employment["Employment Type"].str.split(", ")

unique_job_types= set()
split_job_types.apply(unique_job_types.update)

unique_job_types=list(unique_job_types)

employment_type_data = employment_type_data.join(pd.DataFrame(columns=unique_job_types))

for job_type in unique_job_types:
    employment_type_data[job_type] = employment_type_data["Employment Type"].apply(
        lambda x: 1 if job_type in x else 0
    )

employment_type_data.rename(columns={'nhân viên chính thức': 'Employment Type: Official',
                                      "bán thời gian":"Employment Type: Part-time", 
                                      "thực tập": "Employment Type: Internship", 
                                      "thời vụ/ nghề tự do":"Employment Type: Freelance"}, inplace=True)

In [35]:
employment_type_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26361 entries, 0 to 26360
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Location                     26361 non-null  int64 
 1   Experience                   26361 non-null  int64 
 2   Job Level                    26361 non-null  int64 
 3   Employment Type              26361 non-null  object
 4   Industry                     26361 non-null  object
 5   Welfare                      26361 non-null  object
 6   Job Description              26361 non-null  object
 7   Job Requirements             26361 non-null  object
 8   Other Information            26361 non-null  object
 9   Follower                     25262 non-null  object
 10  Salary                       26361 non-null  object
 11  Employment Type: Internship  26361 non-null  int64 
 12  Employment Type: Part-time   26361 non-null  int64 
 13  Employment Type: Official    26

### Follower

In [36]:
follower_data=employment_type_data.copy()

In [37]:
follower_data["Follower"]=follower_data["Follower"].fillna("0")
follower_data["Follower"]=follower_data["Follower"].str.replace(".","",regex=True)
follower_data["Follower"]=follower_data["Follower"].str.replace("Not specified","0",regex=True)
follower_data["Follower"]=follower_data["Follower"].str.replace("Not Found","0",regex=True)
follower_data["Follower"]=follower_data["Follower"].astype(int)



In [38]:
follower_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26361 entries, 0 to 26360
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Location                     26361 non-null  int64 
 1   Experience                   26361 non-null  int64 
 2   Job Level                    26361 non-null  int64 
 3   Employment Type              26361 non-null  object
 4   Industry                     26361 non-null  object
 5   Welfare                      26361 non-null  object
 6   Job Description              26361 non-null  object
 7   Job Requirements             26361 non-null  object
 8   Other Information            26361 non-null  object
 9   Follower                     26361 non-null  int32 
 10  Salary                       26361 non-null  object
 11  Employment Type: Internship  26361 non-null  int64 
 12  Employment Type: Part-time   26361 non-null  int64 
 13  Employment Type: Official    26

### Salary

In [40]:
salary_data=follower_data.copy()   

In [41]:
# Hàm chuyển đổi chuỗi tiền tệ từ định dạng 'Tr' sang số và lấy giá trị nhỏ nhất
def convert_salary_to_min(salary):
    if(salary != 'cạnh tranh'):   
        if "lên đến"in salary and "usd" in salary:
            if "-" in salary:
                parts = salary.replace("lên đến", "").replace(" usd", "").replace(".","").split(" - ")
                min_salary = float(parts[0].replace(",", ""))*23000
                return min_salary
            else:
                value = salary.replace("trên", "").replace("lên đến", "").replace(" usd", "").replace(".","").strip()
                min_salary = float(value.replace(",", ""))*23000
                return min_salary
        
        elif "trên"in salary and "usd" in salary:
            value = salary.replace("trên", "").replace(" usd", "").replace(".","").strip()
            min_salary = float(value.replace(",", ""))*23000
            return min_salary
        
        elif 'usd' in salary and '-' in salary:
            parts = salary.replace(" usd", "").replace(" tr", "").replace(".","").split(" - ")
            min_salary = float(parts[0].replace(",", ""))*23000
            return min_salary
        
        elif 'usd' in salary:
            min_salary = float(salary.replace(" usd", "").replace(".","").replace(",", ""))*23000
            return min_salary
        
        elif 'trên' in salary and 'tr' in salary:
            if "-" in salary:
                parts = salary.replace("lên đến", "").replace("trên", "").replace(" tr", "").replace(" vnd","").split(" - ")
                min_salary = float(parts[0].replace(",", ".")) * 1_000_000
                return min_salary
            else:
                value = salary.replace("lên đến", "").replace("trên", "").replace(" tr", "").replace(" vnd","").strip()
                min_salary = float(value.replace(",", ".")) * 1_000_000
                return min_salary
        elif 'lên đến' in salary and "tr" in salary:
            if "-" in salary:
                parts = salary.replace("lên đến", "").replace(" tr", "").replace(" vnd","").split(" - ")
                min_salary = float(parts[0].replace(",", ".")) * 1_000_000
                return min_salary
            else:
                value = salary.replace("lên đến", "").replace(" tr", "").replace(" vnd","").strip()
                min_salary = float(value.replace(",", ".")) * 1_000_000
                return min_salary
        
        # Kiểm tra nếu là dạng 'Tr' VND
        elif 'tr' in salary and ' - ' in salary:
            # Loại bỏ " Tr VND", tách và nhân với 1,000,000
            parts = salary.replace(" tr", "").replace(" vnd","").split(" - ")
            min_salary = float(parts[0].replace(",", ".")) * 1_000_000
            return min_salary
        
        # Kiểm tra nếu là dạng 'Tr' VND
        elif 'tr' in salary:
            # Loại bỏ " Tr VND" và nhân với 1,000,000
            min_salary = float(salary.replace(" tr", "").replace(" vnd","")) * 1_000_000
            return min_salary
        
    return "Cạnh tranh"

In [42]:
salary_data['Salary'] = salary_data['Salary'].replace(
    ['0','Thỏa thuận', 'Negotiable', 'Not Specified', 'Not Found', 'not found', 'Not specifield'], 
    np.nan)
salary_data['Salary'] = salary_data['Salary'].str.lower()
salary_data['Salary'] = salary_data['Salary'].apply(convert_salary_to_min)

In [43]:
salary_data

,Location,Experience,Job Level,Employment Type,Industry,Welfare,Job Description,Job Requirements,Other Information,Follower,Salary,Employment Type: Internship,Employment Type: Part-time,Employment Type: Official,Employment Type: Freelance
0,3,3,4,nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Phụ cấp, Đồng phục, Chế độ th...",Tuyển dụng và đào tạo: Quản lý quy trình tuyển...,Trình độ: Tốt nghiệp cao đẳng/đại học chuyên n...,Bằng cấp:\n Đại học\nThời gian ...,0,Cạnh tranh,0,0,1,0
1,3,0,2,nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Phụ cấp, Xe đưa đón, Đồng phụ...","Thương lượng với khách hàng về giá cả, chất lư...","Tốt nghiệp đại học các chuyên ngành Kinh tế, n...",Bằng cấp:\n Đại học\nĐộ tuổi:\n...,0,Cạnh tranh,0,0,1,0
2,70,0,0,not specified,Not Found,"Chế độ bảo hiểm, Phụ cấp, Xe đưa đón, Đào tạo,...",Not specified,Not specified,Bằng cấp:\n Trung cấp\nHình thứ...,0,Cạnh tranh,0,0,0,0
3,21,2,3,nhân viên chính thức,\n ...,"Laptop, Chế độ bảo hiểm, Du Lịch, Phụ cấp, Xe ...",Job Description:\n1、Develop and optimize depar...,Experience Requirement：\n- Over 3 years...,Bằng cấp:\n Đại học\nĐộ tuổi:\n...,169,Cạnh tranh,0,0,1,0
4,2,1,2,nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Du Lịch, Chế độ thưởng, Chăm ...",- Phát triển khách hàng; tìm kiếm khách hàng ...,- Tốt nghiệp Đại Học hệ chính qui thuộc các n...,Độ tuổi:\n Không giới hạn tuổi\...,227,Cạnh tranh,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26356,3,0,1,nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Phụ cấp, Chế độ thưởng, Chăm ...","· Vận hành hệ thống bảo mật, giữ cho hệ ...",Điều kiện ứng tuyển\n· Là sinh viên tốt ...,Bằng cấp:\n Đại học\nĐộ tuổi:\n...,0,34500000.0,0,0,1,0
26357,19,0,1,nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Du Lịch, Phụ cấp, Chế độ thưở...","Với mục tiêu mở rộng thêm các Siêu thị, Team P...",- Nam từ 22 - 30 tuổi. Sức khỏe tốt\n- Không y...,Bằng cấp:\n Cao đẳng\nGiới tính...,20965,Cạnh tranh,0,0,1,0
26358,69,0,1,bán thời gian,\n ...,"Chế độ thưởng, Đào tạo, Tăng lương",- Dạy luyện thi chứng chỉ FE (chứng chỉ chuẩn ...,- Đã có chứng chỉ FE.\n- Ứng viên cần có kiến ...,Độ tuổi:\n Không giới hạn tuổi\...,98,7000000.0,0,1,0,0
26359,16,0,1,nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Du Lịch, Phụ cấp, Chế độ thưở...","Với mục tiêu mở rộng thêm các Siêu thị, Team P...",- Nam từ 22 - 30 tuổi. Sức khỏe tốt\n- Không y...,Bằng cấp:\n Cao đẳng\nGiới tính...,20965,Cạnh tranh,0,0,1,0


In [47]:
salary_data.to_csv('../dataset/pre_translate_data.csv',index=False,encoding='utf-8-sig')    

#### Đọc dữ liệu

In [ ]:
raw_data_path='../dataset/pre_translate_data/pre_translate_data.csv'

raw_data=pd.read_csv(raw_data_path)

In [ ]:
raw_data.head(5)

#### Chuyển cột Employment type sang tiếng anh

In [ ]:
# Tạo từ điển translation_dict
employmentType_translation_dict = {
    'thực tập': 'internship',
    'bán thời gian': 'part-time',
    'nhân viên chính thức': 'official',
    'thời vụ/ nghề tự do': 'freelance',
}

def translate_using_dict(text, translation_dict):
    for key, value in translation_dict.items():
        text = text.replace(key, value)
    return text

# Áp dụng hàm vào cột 'Employment Type'
raw_data['Employment Type'] = raw_data['Employment Type'].apply(lambda x: translate_using_dict(x, employmentType_translation_dict))

In [ ]:
raw_data.head(5)

#### Tách các ngành trong Industry và trả về danh sách các ngành

In [ ]:
# Hàm để tách các ngành trong chuỗi và trả về một danh sách ngành
def extract_type(text):
    # Tách các ngành ra bằng dấu phẩy và loại bỏ khoảng trắng
    types = [_type.strip() for _type in text.split(',')]
    return types

# Áp dụng hàm vào cột 'Industry' và nới rộng thành một danh sách các ngành
all_industries = raw_data['Industry'].apply(extract_type).explode()

# Đếm số lần xuất hiện của từng ngành
industry_counts = all_industries.value_counts()

# Hiển thị kết quả
print((industry_counts))
# Lấy danh sách tên các ngành từ industry_counts
industry_names = industry_counts.index.tolist()

# In danh sách các ngành
print(industry_names)

#### Tạo từ điển các ngành bằng tiếng anh và áp dụng vào dữ liệu

In [ ]:
industry_translation_dict = {
    'Bán hàng / Kinh doanh': 'sales_business',
    'Ngân hàng': 'banking',
    'Dịch vụ khách hàng': 'customer_service',
    'Tiếp thị / Marketing': 'marketing',
    'Bán lẻ / Bán sỉ': 'retail_wholesale',
    'Tài chính / Đầu tư': 'finance_investment',
    'Kế toán / Kiểm toán': 'accounting_auditing',
    'Điện / Điện tử / Điện lạnh': 'electricity_electronics_refrigeration',
    'Tư vấn': 'consulting',
    'Sản xuất / Vận hành sản xuất': 'manufacturing_operations',
    'Cơ khí / Ô tô / Tự động hóa': 'mechanical_automotive_automation',
    'Xây dựng': 'construction',
    'Hành chính / Thư ký': 'administrative_secretarial',
    'Giáo dục / Đào tạo': 'education_training',
    'CNTT - Phần mềm': 'it_software',
    'Bất động sản': 'real_estate',
    'Nhân sự': 'human_resources',
    'Quản lý điều hành': 'executive_management',
    'Y tế / Chăm sóc sức khỏe': 'healthcare_medical',
    'Vận chuyển / Giao nhận /  Kho vận': 'transportation_logistics_warehouse',
    'Thực phẩm & Đồ uống': 'food_beverages',
    'Dệt may / Da giày / Thời trang': 'textile_leather_fashion',
    'Bảo trì / Sửa chữa': 'maintenance_repair',
    'Quản lý chất lượng (QA/QC)': 'quality_management_qaqc',
    'Tiếp thị trực tuyến': 'online_marketing',
    'Xuất nhập khẩu': 'import_export',
    'Not Found': 'not_found',
    'Quảng cáo / Đối ngoại / Truyền Thông': 'advertising_public_relations_media',
    'Dược phẩm': 'pharmaceuticals',
    'Biên phiên dịch': 'translation_interpretation',
    'Nhà hàng / Khách sạn': 'restaurant_hotel',
    'Mỹ thuật / Nghệ thuật / Thiết kế': 'fine_arts_art_design',
    'Kiến trúc': 'architecture',
    'Luật / Pháp lý': 'law_legal',
    'Bảo hiểm': 'insurance',
    'Ngành khác': 'other_industries',
    'Thu mua / Vật tư': 'procurement_supplies',
    'CNTT - Phần cứng / Mạng': 'it_hardware_networking',
    'Bưu chính viễn thông': 'postal_telecommunications',
    'Nội ngoại thất': 'interior_exterior_design',
    'Mới tốt nghiệp / Thực tập': 'new_graduates_internship',
    'Du lịch': 'tourism',
    'Truyền hình / Báo chí / Biên tập': 'television_journalism_editing',
    'Hóa học': 'chemistry',
    'Lao động phổ thông': 'manual_labor',
    'Công nghệ thực phẩm / Dinh dưỡng': 'food_technology_nutrition',
    'Chứng khoán': 'securities',
    'Môi trường': 'environment',
    'Hàng gia dụng / Chăm sóc cá nhân': 'household_goods_personal_care',
    'Công nghệ sinh học': 'biotechnology',
    'An toàn lao động': 'occupational_safety',
    'Đồ gỗ': 'wooden_goods',
    'Tổ chức sự kiện': 'event_organization',
    'Nông nghiệp': 'agriculture',
    'Thống kê': 'statistics',
    'In ấn / Xuất bản': 'printing_publishing',
    'Giải trí': 'entertainment',
    'An Ninh / Bảo Vệ': 'security_protection',
    'Dầu khí': 'oil_gas',
    'Hàng không': 'aviation',
    'Hàng hải': 'maritime',
    'Chăn nuôi / Thú y': 'livestock_veterinary',
    'Thủy sản / Hải sản': 'aquaculture_seafood',
    'Trắc địa / Địa Chất': 'surveying_geology',
    'Khoáng sản': 'minerals',
    'Lâm Nghiệp': 'forestry',
    'Phi chính phủ / Phi lợi nhuận': 'ngo_non_profit',
    'Thủy lợi': 'irrigation',
    'Thư viện': 'library'
}
raw_data['Industry'] = raw_data['Industry'].apply(lambda x: translate_using_dict(x, industry_translation_dict))

#### Onehot Encoding cho cột Industry

In [ ]:
# Tạo một DataFrame chứa mỗi giá trị ngành là một cột với one-hot encoding
one_hot = raw_data['Industry'].apply(extract_type).explode().str.get_dummies().groupby(level=0).sum()

# Đổi tên cột theo định dạng Industry: [industry]
one_hot.columns = ['Industry: ' + col for col in one_hot.columns]
one_hot.head(5)

In [ ]:
raw_data = pd.concat([raw_data, one_hot], axis=1)

#### Lọc ra các chế độ bảo hiểm Welfare

In [ ]:
all_welfares = raw_data['Welfare'].apply(extract_type).explode()

# Đếm số lần xuất hiện của từng ngành
welfare_counts = all_welfares.value_counts()

# Hiển thị kết quả
print((welfare_counts))
# Lấy danh sách tên các ngành từ industry_counts
welfare_names = welfare_counts.index.tolist()

# In danh sách các ngành
print(welfare_names)

In [ ]:
welfare_translation_dict = {
    'Chế độ thưởng': 'bonus',
    'Chế độ bảo hiểm': 'insurance',
    'Đào tạo': 'training',
    'Tăng lương': 'salary_increase',
    'Chăm sóc sức khỏe': 'healthcare',
    'Du Lịch': 'travel',
    'Nghỉ phép năm': 'annual_leave',
    'Phụ cấp': 'allowance',
    'Đồng phục': 'uniform',
    'Công tác phí': 'business_trip_expenses',
    'Laptop': 'laptop',
    'Phụ cấp thâm niên': 'seniority_allowance',
    'CLB thể thao': 'sports_club',
    'Du lịch nước ngoài': 'overseas_travel',
    'Xe đưa đón': 'shuttle_service',
    'Not specified': 'not_specified'
}
raw_data['Welfare'] = raw_data['Welfare'].apply(lambda x: translate_using_dict(x, welfare_translation_dict))

#### Onehot encoding cho Welfare

In [ ]:
# Tạo một DataFrame chứa mỗi giá trị welfare là một cột với one-hot encoding
welfare_one_hot = raw_data['Welfare'].apply(extract_type).explode().str.get_dummies().groupby(level=0).sum()

# Đổi tên cột theo định dạng Welfare: [welfare]
welfare_one_hot.columns = ['Welfare: ' + col for col in welfare_one_hot.columns]

# Hiển thị kết quả
welfare_one_hot.head(5)

In [ ]:
raw_data = pd.concat([raw_data, welfare_one_hot], axis=1)

#### Loại bỏ các kí tự không cần thiết trong Industry, Welfare

In [ ]:
raw_data['Industry'] = raw_data['Industry'].str.replace('\n', '', regex=False)
raw_data['Welfare'] = raw_data['Welfare'].str.replace(' ', '', regex=False)
raw_data['Industry'] = raw_data['Industry'].str.replace(' ', '', regex=False)

#### Onehot encoding cho Location

In [ ]:
number_to_city = {
  0: 'Hai Duong',
  1: 'Long An',
  2: 'Ha Noi',
  3: 'Ho Chi Minh',
  4: 'Binh Duong',
  5: 'Đong Nai',
  6: 'Quang Ngai',
  7: 'Vinh Long',
  8: 'Lam Đong',
  9: 'Can Tho',
  10: 'Bac Ninh',
  11: 'Dak Lak',
  12: 'Gia Lai',
  13: 'Bac Giang',
  14: 'Binh Thuan',
  15: 'Tay Ninh',
  16: 'Ba Ria - Vung Tau',
  17: 'Ha Nam',
  18: 'Thai Binh',
  19: 'Bac Lieu',
  20: 'Thanh Hoa',
  21: 'Binh Đinh',
  22: 'Tra Vinh',
  23: 'Hoa Binh',
  24: 'Đa Nang',
  25: 'Hung Yen',
  26: 'Hai Phong',
  27: 'Kien Giang',
  28: 'Vinh Phuc',
  29: 'Ha Giang',
  30: 'Tuyen Quang',
  31: 'Đien Bien',
  32: 'Nam Dinh',
  33: 'Lang Son',
  34: 'Nghe An',
  35: 'Quang Tri',
  36: 'Quang Ninh',
  37: 'Ninh Thuan',
  38: 'Đong Thap',
  39: 'Quang Nam',
  40: 'Thua Thien- Hue',
  41: 'Tien Giang',
  42: 'Khanh Hoa',
  43: 'Phu Yen',
  44: 'Quang Binh',
  45: 'Hau Giang',
  46: 'Binh Phuoc',
  47: 'Phu Tho',
  48: 'Ninh Binh',
  49: 'Thai Nguyen',
  50: 'Bac Can',
  51: 'Son La',
  52: 'Ha Tinh',
  53: 'Yen Bai',
  54: 'Ca Mau',
  55: 'An Giang',
  56: 'Ben Tre',
  57: 'Lao Cai',
  58: 'Soc Trang',
  59: 'Cao Bang',
  60: 'Dak Nong',
  61: 'Kon Tum',
  62: 'Lai Chau',
  63: 'Nam Đinh',
  64: 'Kv Đong Nam Bo',
  65: 'Kv Tay Nguyen',
  66: 'Kv Nam Trung Bo',
  67: 'Kv Bac Trung Bo',
  68: 'Đong Bang Song Cuu Long',
  69: 'Toan Quoc',
  70: 'Not Specified',
  71: 'Svay Rieng',
  72: 'Tokyo',
  73: 'Xiangkhouang',
  74: 'Champasak',
  75: 'Yokohama',
  76: 'Hokkaido',
  77: 'Malaysia',
  78: 'Attapeu',
  79: 'Bangkok',
  80: 'Kratie',
  81: 'Phnompenh',
  82:"Vientiane",
  83:"Kuala Lumpur",
  84:"Quoc Te"
}

raw_data['Location'] = raw_data['Location'].map(number_to_city)

In [ ]:
# Áp dụng One-Hot Encoding cho cột 'Location'
onehot_encoded = pd.get_dummies(raw_data['Location'], prefix='Location')

# Chuyển giá trị True/False thành 1/0
onehot_encoded = onehot_encoded.astype(int)

# Thêm các cột One-Hot vào DataFrame gốc
raw_data = pd.concat([raw_data, onehot_encoded], axis=1)

# Kiểm tra kết quả
raw_data.head()

In [ ]:
# Danh sách các cột cần loại bỏ
columns_to_drop = ['Industry', 'Welfare', 'Location', 'Employment Type']

# Loại bỏ các cột
raw_data = raw_data.drop(columns=columns_to_drop)

#### Chuyển tất cả giá trị của 3 cột Job Description, Job Requirements, Other Information về dạng chữ thường

In [ ]:
raw_data['Job Description'] = raw_data['Job Description'].astype(str).str.lower()
raw_data['Job Requirements'] = raw_data['Job Requirements'].astype(str).str.lower()
raw_data['Other Information'] = raw_data['Other Information'].astype(str).str.lower()

#### Tạo cột mới Language requirement, xác định xem ngoại ngữ có được yêu cầu cho công việc hay không

In [ ]:
# Define the `language_keywords`
language_keywords = [
    # Tiếng Việt
    'tiếng anh', 'tiếng trung', 'tiếng nhật', 'tiếng hàn', 'tiếng pháp', 'tiếng đức', 'tiếng nga', 'tiếng tây ban nha',
    'tiếng ý', 'tiếng bồ đào nha', 'tiếng ả rập', 'tiếng thái', 'tiếng việt', 'tiếng ấn độ', 'tiếng ba lan', 'tiếng thụy điển',
    'tiếng hà lan', 'tiếng na uy', 'tiếng phạn', 'tiếng la tinh', 'tiếng hy lạp', 'tiếng do thái', 'tiếng séc', 'tiếng hungary', 'nước ngoài',
    
    'kỹ năng ngoại ngữ', 'trình độ ngoại ngữ', 'chứng chỉ ngoại ngữ', 'giao tiếp quốc tế', 'giao tiếp đa văn hóa',
    'phiên dịch viên', 'biên dịch viên', 'giảng viên ngôn ngữ', 'giáo viên ngoại ngữ',
    'chuyên gia ngôn ngữ', 'nhà ngôn ngữ học', 'thông dịch viên', 'giảng dạy ngoại ngữ',
    'toeic', 'ielts', 'toefl', 'hsk', 'jlpt', 'topik', 'delf', 'dalf', 'testdaf', 'torfl',

    # English keywords
    'english', 'chinese', 'japanese', 'korean', 'french', 'german', 'russian', 'spanish', 'language', 'communication',
    'foreign language', 'multilingual',
    'language certification', 'language skills', 'bilingual', 'trilingual', 'native english speaker'
]


# Define a function to check for language keywords in the text
def has_language_requirements(text):
    text = text.lower()  # Convert text to lowercase
    # Check if any keyword is in the text
    return 1 if any(keyword in text for keyword in language_keywords) else 0

# Apply the function directly to create the new column
raw_data['Language requirement'] = raw_data['Job Requirements'].astype(str).apply(has_language_requirements)

#### Tạo cột bằng cấp (Education Levels) xác định bằng cấp được yêu cầu

In [ ]:
# Định nghĩa từ điển trình độ học vấn
education_levels_dict = {
    # Tiếng Việt
    'trung học': 1,
    'trung cấp': 2,
    'cao đẳng': 3,
    'đại học': 4,
    'sau đại học': 5,
    
    # Tiếng Anh
    'college': 3,
    'university': 4,
    'bachelor’s degree': 4,
    'master': 5,
    'phd': 5,
}

# Hàm tìm kiếm trình độ học vấn từ các cột
def get_education_level(row):
    # Kiểm tra trong cột 'Other Information' theo thứ tự từ điển
    for keyword, level in education_levels_dict.items():
        if keyword.lower() in row['Other Information'].lower():
            return level
    
    # Nếu không tìm thấy trong 'Other Information', kiểm tra trong 'Job Requirements'
    for keyword, level in education_levels_dict.items():
        if keyword.lower() in row['Job Requirements'].lower():
            return level
    
    # Nếu không tìm thấy trình độ học vấn, trả về NaN hoặc 0
    return 0  # hoặc có thể là NaN nếu bạn muốn

# Áp dụng hàm trên để tạo cột 'Education Levels'
raw_data['Education Levels'] = raw_data.apply(get_education_level, axis=1)

#### Dịch cột Job Requirements sang tiếng anh

In [ ]:
#raw_data = parallelize_column(raw_data, column_name='Job Requirements')

In [ ]:
raw_data_path='raw_Data_new.csv'

raw_data_neww=pd.read_csv(raw_data_path)
raw_data['Job Requirements'] = raw_data_neww['Job Requirements']

#### Tạo cột mới Gender Requirement 

In [ ]:
# Tạo một hàm để kiểm tra nội dung trong cột 'Job Requirements'
def detect_gender(job_requirements):
    job_requirements_lower = str(job_requirements).lower()
    if 'male' in job_requirements_lower and 'female' in job_requirements_lower and job_requirements_lower.count('male') > 1:
        return 'both'
    elif 'female' in job_requirements_lower:
        return 'female'
    elif 'male' in job_requirements_lower:
        return 'male'
    else:
        return 'both'

# Áp dụng hàm để tạo cột 'Gender'
raw_data['Gender Requirement'] = raw_data['Job Requirements'].apply(detect_gender)

#### Onehot Encoding cho cột Gender Requirement

In [ ]:
# Thực hiện One-Hot Encoding cho cột 'Gender Requirement'
raw_data = pd.get_dummies(raw_data, columns=['Gender Requirement'], prefix='Gender_requirement')
raw_data = raw_data.replace({True: 1, False: 0})
# Chuyển đổi giá trị True/False thành 1/0

#### Sắp xếp, Loại bỏ các cột đã xử lí và lưu dữ liệu

In [ ]:
# Danh sách các cột cần loại bỏ
columns_to_drop = ['Job Requirements', 'Other Information', 'Job Description']

# Loại bỏ các cột
raw_data = raw_data.drop(columns=columns_to_drop)

columns = raw_data.columns.tolist()
columns.remove('Education Levels')
columns.insert(3, 'Education Levels')
# Di chuyển 'Salary' vào vị trí cuối cùng
columns.remove('Salary')
columns.append('Salary')

# Sắp xếp lại DataFrame với thứ tự cột mới
raw_data = raw_data[columns]

In [ ]:
raw_data_define_salary = raw_data[raw_data['Salary'] != 'Cạnh tranh']
raw_data_undefine_salary = raw_data[raw_data['Salary'] == 'Cạnh tranh']

raw_data_define_salary.to_csv('../dataset/Defined_salary.csv', index=False,encoding='utf-8-sig')
raw_data_undefine_salary.to_csv('../dataset/Undefined_salary.csv', index=False,encoding='utf-8-sig')